<a href="https://colab.research.google.com/github/chewzzz1014/fyp/blob/master/ner/src/train_ner_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Train NER Models

In [1]:
# mount drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!mkdir spacy_ner_data

In [ ]:
import json
import random
from sklearn.model_selection import train_test_split
import spacy
from spacy.tokens import DocBin

# Load JSON data
with open('/content/drive/MyDrive/FYP/Implementation/Resume Dataset/200_resumes_annotated.json', "r") as f:
    data = json.load(f)

def remove_overlapping_entities(entities):
    """Remove overlapping entities from the list."""
    entities = sorted(entities, key=lambda x: x[0])  # Sort by start position
    non_overlapping = []
    last_end = -1
    for start, end, label in entities:
        if start >= last_end:  # Only add if there's no overlap with the previous entity
            non_overlapping.append((start, end, label))
            last_end = end
    return non_overlapping

# Function to convert JSON data to Spacy's DocBin format
def convert_to_spacy_format(data):
    nlp = spacy.blank("en")  # Load a blank Spacy model
    doc_bin = DocBin()  # Container for our docs

    for item in data:
        text = item['data']['Text']  # Full document text
        entities = []

        for annotation in item['annotations'][0]['result']:
            start = annotation['value']['start']
            end = annotation['value']['end']
            label = annotation['value']['labels'][0]  # Entity label
            entities.append((start, end, label))

        entities = remove_overlapping_entities(entities)  # Remove overlapping entities
        # Create a Spacy doc and add entities to it
        doc = nlp.make_doc(text)
        spans = [doc.char_span(start, end, label=label) for start, end, label in entities]
        # Filter out None spans if Spacy can't align the character indices with tokens
        spans = [span for span in spans if span is not None]
        doc.ents = spans  # Assign entities to the doc
        doc_bin.add(doc)

    return doc_bin

# Split data into train and test sets
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Convert train and test sets to Spacy format
train_doc_bin = convert_to_spacy_format(train_data)
test_doc_bin = convert_to_spacy_format(test_data)

# Save the train and test data to .spacy files
train_doc_bin.to_disk("spacy_ner_data/train_data.spacy")
test_doc_bin.to_disk("spacy_ner_data/test_data.spacy")

## Spacy NER

In [ ]:
# create base_config.cfg and paste the config generated from spacy widget
# update train and test file path
!touch base_config.cfg

In [ ]:
# generate config.cfg from base_config.cfg
!python -m spacy init fill-config base_config.cfg config.cfg

✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
!python -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 2.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
# train model using hyperparameters set in config.cfg
# save trained model in spacy-output/ dir
!python -m spacy train config.cfg --output ./spacy_output
!cp -r ./spacy_output /content/drive/MyDrive/FYP/Implementation/

In [ ]:
# evaluate trained model performance
# store output and visualization into result/ dir
!python -m spacy evaluate spacy_output/model-best spacy_ner_data/test_data.spacy -dp spacy_output

ℹ Using CPU

================================== Results ==================================

TOK     100.00
NER P   51.12 
NER R   41.26 
NER F   45.66 
SPEED   2395  


=============================== NER (per type) ===============================

                P       R       F
NAME        89.66   78.79   83.87
JOB         72.00   32.43   44.72
DEG         62.16   63.89   63.01
UNI         38.89   34.15   36.36
EMAIL       63.33   95.00   76.00
LOC         39.39   31.71   35.14
WORK PER    75.45   83.00   79.05
COMPANY     28.42   36.49   31.95
SKILL       40.96   28.96   33.93
PHONE       89.66   83.87   86.67
STUDY PER   65.62   58.33   61.76

<IPython.core.display.HTML object>
Traceback (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.10/dist-packages/spacy/__main__.p

In [ ]:
# make prediction
import spacy
resume_text = '''
John Doe lives at 1234 Elm Street in Los Angeles, CA 90001. He can be reached at +1 (555) 123-4567 or via email at john.doe@example.com. John is a results-driven software engineer with over 5 years of experience in web development and cloud infrastructure, with strong knowledge of JavaScript, Python, and cloud technologies like AWS and Azure. Currently, he works as a Software Engineer at Google LLC in San Francisco, CA, where he has been employed since August 2019. In this role, he has developed scalable web applications using JavaScript, Node.js, and React, deployed and maintained cloud infrastructure on AWS, reducing downtime by 20%, and led a team of 4 engineers to enhance backend performance by 30%. Previously, he worked as a Junior Developer at Tech Innovators Inc. in Austin, TX, from July 2017 to July 2019, where he created RESTful APIs using Python and Flask, collaborated with front-end developers to build and deploy user-facing applications, and wrote unit and integration tests, improving code coverage by 15%.

John holds a Master of Science in Computer Science from the University of California, Berkeley, with a graduation date of May 2017, and a Bachelor of Science in Information Technology from the University of Texas at Austin, graduated in May 2015. His skillset includes proficiency in programming languages like Python, JavaScript, and Java; frameworks such as React, Flask, and Django; cloud platforms including AWS, Google Cloud, and Azure; as well as other tools like Git, Docker, Kubernetes, and SQL. He is certified as an AWS Certified Solutions Architect – Associate, earned in 2020, and as a Google Professional Cloud Architect, earned in 2021'
'''
nlp = spacy.load("spacy-output/model-best")
doc = nlp(resume_text.lower())

print(doc.ents)

for ent in doc.ents:
    print(f"{ent.text}: {ent.label_}")

(john doe, in los, (555) 123-4567, john.doe@example.com, john is, aws, restful apis, master of science, bachelor of science in information technology, python, aws, azure, git, docker)
john doe: NAME
in los: LOC
(555) 123-4567: PHONE
john.doe@example.com: EMAIL
john is: NAME
aws: SKILL
restful apis: SKILL
master of science: DEG
bachelor of science in information technology: DEG
python: SKILL
aws: SKILL
azure: SKILL
git: SKILL
docker: SKILL


In [ ]:
from spacy import displacy
displacy.render(doc, style="ent", jupyter=True)

## Flair NER

In [2]:
!pip install flair

In [1]:
# convert spacy data into flair data
import spacy
from spacy.tokens import DocBin
import os

def convert_spacy_to_flair(input_file, output_file):
    """
    Convert SpaCy binary format to Flair's CoNLL format.

    Args:
        input_file (str): Path to SpaCy binary file (.spacy)
        output_file (str): Path to output file for Flair format
    """
    # Load spaCy model
    nlp = spacy.blank("en")

    # Load the DocBin
    doc_bin = DocBin().from_disk(input_file)
    docs = list(doc_bin.get_docs(nlp.vocab))

    with open(output_file, 'w', encoding='utf-8') as f:
        for doc in docs:
            tokens = [(t.text, t.ent_iob_, t.ent_type_) for t in doc]

            # Write tokens in CoNLL format
            for token in tokens:
                text, iob, ent_type = token

                # Convert spaCy IOB to CoNLL format
                if iob == 'O':
                    tag = 'O'
                else:
                    tag = f'{iob}-{ent_type}' if ent_type else 'O'

                # Write line: token and NER tag
                f.write(f'{text} {tag}\n')

            # Empty line between sentences
            f.write('\n')

def convert_spacy_json_to_flair(input_file, output_file):
    """
    Convert SpaCy JSON format to Flair's CoNLL format.

    Args:
        input_file (str): Path to JSON file with SpaCy annotations
        output_file (str): Path to output file for Flair format
    """
    import json

    nlp = spacy.blank("en")

    with open(input_file, 'r', encoding='utf-8') as f:
        training_data = json.load(f)

    with open(output_file, 'w', encoding='utf-8') as f:
        for example in training_data:
            text = example['text']
            ents = example.get('entities', [])

            # Create a spaCy doc
            doc = nlp(text)

            # Add entities to doc
            spans = []
            for start, end, label in ents:
                span = doc.char_span(start, end, label=label)
                if span is not None:
                    spans.append(span)
            doc.ents = spans

            # Convert to CoNLL format
            tokens = [(t.text, t.ent_iob_, t.ent_type_) for t in doc]

            for token in tokens:
                text, iob, ent_type = token
                if iob == 'O':
                    tag = 'O'
                else:
                    tag = f'{iob}-{ent_type}' if ent_type else 'O'
                f.write(f'{text} {tag}\n')

            f.write('\n')

# Example usage for JSON format
flair_train_json = "flair_train.txt"
flair_test_json = "flair_test.txt"

convert_spacy_to_flair('/content/spacy_ner_data/train_data.spacy', flair_train_json)
convert_spacy_to_flair('/content/spacy_ner_data/test_data.spacy', flair_test_json)

FileNotFoundError: [Errno 2] No such file or directory: '/content/spacy_ner_data/train_data.spacy'

In [ ]:
# convert spacy data into flair data
import spacy
from spacy.training import Corpus

!python -m spacy download de_core_news_sm
nlp = spacy.load("de_core_news_sm")
corpus = Corpus("/content/spacy_ner_data/test_data.spacy")

data = corpus(nlp)

# Flair supports BIO and BIOES, see https://github.com/flairNLP/flair/issues/875
def rename_biluo_to_bioes(old_tag):
    new_tag = ""
    try:
        if old_tag.startswith("L"):
            new_tag = "E" + old_tag[1:]
        elif old_tag.startswith("U"):
            new_tag = "S" + old_tag[1:]
        else:
            new_tag = old_tag
    except:
        pass
    return new_tag


def generate_corpus():
    corpus = []
    n_ex = 0
    for example in data:
        n_ex += 1
        text = example.text
        doc = nlp(text)
        tags = example.get_aligned_ner()
        # Check if it's an empty list of NER tags.
        if None in tags:
            pass
        else:
            new_tags = [rename_biluo_to_bioes(tag) for tag in tags]
            for token, tag in zip(doc,new_tags):
                row = token.text +' '+ token.pos_ +' ' +tag + '\n'
                corpus.append(row)
            corpus.append('\n')
    return corpus

def write_file(filepath):
    with open(filepath, 'w', encoding='utf-8') as f:
        corpus = generate_corpus()
        f.writelines(corpus)

def main():
    write_file('flair_test.txt')

main()

  Using cached https://github.com/explosion/spacy-models/releases/download/de_core_news_sm-3.7.0/de_core_news_sm-3.7.0-py3-none-any.whl (14.6 MB)
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [3]:
# convert json into flair data
import json
import random
from typing import List, Dict, Tuple
import spacy
from collections import defaultdict

class NERConverter:
    def __init__(self):
        self.nlp = spacy.load("en_core_web_sm")

    def get_bioes_label(self, token_index: int, entity_length: int, current_position: int, label: str) -> str:
        """
        Convert to BIOES format
        - S-: Single token entity
        - B-: Beginning of multi-token entity
        - I-: Inside of multi-token entity
        - E-: End of multi-token entity
        - O: Outside
        """
        if entity_length == 1:
            return f'S-{label}'
        if current_position == 0:
            return f'B-{label}'
        if current_position == entity_length - 1:
            return f'E-{label}'
        return f'I-{label}'

    def convert_to_bioes_format(self, json_data: List[dict]) -> List[List[Tuple[str, str]]]:
        """Convert JSON annotations to BIOES format."""
        all_sentences = []

        for item in json_data:
            text = item['data']['Text']
            doc = self.nlp(text)

            # Initialize character-level labels
            char_labels = ['O'] * len(text)

            # First pass: identify entity boundaries and lengths
            entity_spans = []
            if item['annotations'] and len(item['annotations']) > 0:
                for ann in item['annotations'][0]['result']:
                    if 'value' in ann:
                        start = ann['value']['start']
                        end = ann['value']['end']
                        label = ann['value']['labels'][0]
                        entity_spans.append((start, end, label))

            # Sort spans by start position
            entity_spans.sort(key=lambda x: x[0])

            # Second pass: apply BIOES labels
            for start, end, label in entity_spans:
                # Get tokens that are part of this entity
                entity_text = text[start:end]
                entity_doc = self.nlp(entity_text)
                entity_length = len([token for token in entity_doc if not token.is_space])

                # Set labels for the entire span
                current_token_idx = 0
                for i in range(start, end):
                    if i == start or text[i-1].isspace():
                        char_labels[i] = self.get_bioes_label(i, entity_length, current_token_idx, label)
                        current_token_idx += 1
                    else:
                        char_labels[i] = char_labels[i-1]

            # Convert to token-level labels
            current_sentence = []
            for sent in doc.sents:
                for token in sent:
                    # Get the most common label for the token's characters
                    token_chars_labels = char_labels[token.idx:token.idx + len(token.text)]
                    label_counts = defaultdict(int)
                    for char_label in token_chars_labels:
                        label_counts[char_label] += 1

                    token_label = max(label_counts.items(), key=lambda x: x[1])[0]
                    current_sentence.append((token.text, token_label))

                if current_sentence:
                    all_sentences.append(current_sentence)
                    current_sentence = []

        return all_sentences

    def write_flair_file(self, sentences: List[List[Tuple[str, str]]], filename: str):
        """Write sentences in BIOES format to file."""
        with open(filename, 'w', encoding='utf-8') as f:
            for sentence in sentences:
                for token, label in sentence:
                    f.write(f'{token} {label}\n')
                f.write('\n')

    def convert_and_split(self, json_data: List[dict], train_file: str, test_file: str, test_ratio: float = 0.2):
        """Convert JSON to BIOES format and split into train/test sets."""
        all_sentences = self.convert_to_bioes_format(json_data)

        # Shuffle and split
        random.shuffle(all_sentences)
        split_idx = int(len(all_sentences) * (1 - test_ratio))

        train_sentences = all_sentences[:split_idx]
        test_sentences = all_sentences[split_idx:]

        # Write to files
        self.write_flair_file(train_sentences, train_file)
        self.write_flair_file(test_sentences, test_file)

        return len(train_sentences), len(test_sentences)

def main():
    # Load JSON data
    with open('/content/drive/MyDrive/FYP/Implementation/Resume Dataset/200_resumes_annotated.json', 'r', encoding='utf-8') as f:
        json_data = json.load(f)

    # Convert and split data
    converter = NERConverter()
    train_count, test_count = converter.convert_and_split(
        json_data,
        train_file='flair_train.txt',
        test_file='flair_test.txt',
        test_ratio=0.2
    )

    print(f'Created {train_count} training sentences and {test_count} test sentences')

main()

Created 298 training sentences and 75 test sentences


In [4]:
from flair.data import Corpus
from flair.datasets import ColumnCorpus

# Define columns for CoNLL (0: word, 1: label)
columns = {0: 'text', 1: 'ner'}

# Set data folder and file names
data_folder = './'
train_file = 'flair_train.txt'
test_file = 'flair_test.txt'

# Load the corpus
corpus: Corpus = ColumnCorpus(data_folder, columns,
                              train_file=train_file,
                              test_file=test_file,
                              dev_file=None)

2024-11-05 13:48:12,229 Reading data from .
2024-11-05 13:48:12,230 Train: flair_train.txt
2024-11-05 13:48:12,233 Dev: None
2024-11-05 13:48:12,234 Test: flair_test.txt
2024-11-05 13:48:13,386 No dev split found. Using 10% (i.e. 30 samples) of the train split as dev data


In [5]:
tag_dictionary = corpus.make_label_dictionary(label_type='ner')
print("Labels:", tag_dictionary.get_items())

2024-11-05 13:48:24,830 Computing label dictionary. Progress:


0it [00:00, ?it/s]
268it [00:00, 8598.17it/s]

2024-11-05 13:48:24,874 Dictionary created for label 'ner' with 11 values: SKILL (seen 2343 times), JOB (seen 543 times), WORK (seen 470 times), COMPANY (seen 381 times), LOC (seen 237 times), UNI (seen 149 times), NAME (seen 146 times), PHONE (seen 144 times), DEG (seen 144 times), STUDY (seen 125 times), EMAIL (seen 112 times)
Labels: ['SKILL', 'JOB', 'WORK', 'COMPANY', 'LOC', 'UNI', 'NAME', 'PHONE', 'DEG', 'STUDY', 'EMAIL']


In [6]:
from collections import Counter

def count_labels(file_path):
    with open(file_path, 'r') as file:
        labels = [line.split()[-1] for line in file if line.strip()]
    return Counter(labels)

print("Train label distribution:", count_labels('flair_train.txt'))
print("Test label distribution:", count_labels('flair_test.txt'))

Train label distribution: Counter({'O': 70550, 'S-SKILL': 1494, 'PER': 1185, 'B-SKILL': 992, 'E-SKILL': 989, 'E-JOB': 532, 'B-JOB': 525, 'E-COMPANY': 338, 'B-COMPANY': 335, 'I-JOB': 296, 'I-DEG': 254, 'I-COMPANY': 242, 'I-SKILL': 197, 'E-UNI': 163, 'B-UNI': 162, 'B-NAME': 158, 'E-NAME': 158, 'S-LOC': 156, 'E-DEG': 154, 'B-DEG': 153, 'E-PHONE': 132, 'B-PHONE': 130, 'I-UNI': 126, 'S-EMAIL': 121, 'B-LOC': 102, 'E-LOC': 102, 'I-PHONE': 83, 'S-COMPANY': 67, 'S-JOB': 52, 'S-PHONE': 22, 'I-NAME': 7, 'S-DEG': 6, 'I-LOC': 3, 'S-UNI': 2, 'B-EMAIL': 1, 'E-EMAIL': 1})
Test label distribution: Counter({'O': 18543, 'PER': 326, 'S-SKILL': 297, 'B-SKILL': 221, 'E-SKILL': 220, 'E-JOB': 127, 'B-JOB': 122, 'E-COMPANY': 89, 'B-COMPANY': 87, 'I-DEG': 80, 'I-COMPANY': 73, 'I-JOB': 61, 'B-UNI': 52, 'E-UNI': 52, 'S-LOC': 49, 'B-DEG': 48, 'E-DEG': 48, 'I-UNI': 45, 'I-SKILL': 39, 'B-NAME': 33, 'E-NAME': 33, 'S-EMAIL': 27, 'B-LOC': 26, 'E-LOC': 26, 'B-PHONE': 25, 'E-PHONE': 25, 'S-JOB': 23, 'I-PHONE': 23, 'S-COM

In [7]:
# create NER tagger
from flair.embeddings import WordEmbeddings, StackedEmbeddings, TransformerWordEmbeddings, FlairEmbeddings
from flair.models import SequenceTagger

# using LSTM-CRF on top of frozen embeddings
# combine flair and glove embeddings
# embeddings = StackedEmbeddings([
#                 WordEmbeddings('glove'),
#                 FlairEmbeddings('news-forward'),
#                 FlairEmbeddings('news-backward'),
#             ])
# tagger = SequenceTagger(hidden_size=256,
#                          embeddings=embeddings,
#                          tag_dictionary=tag_dictionary,
#                          tag_type='ner',
#                          use_crf=True,
#                          tag_format = 'BIOES')

# using transformer embedding
# embeddings = TransformerWordEmbeddings('bert-base-uncased',
#                                       fine_tune=True,
#                                       layers='-1',
#                                       subtoken_pooling='first')
embeddings = TransformerWordEmbeddings(
    'roberta-base',  # or 'bert-base-uncased'
    fine_tune=True,
    layers='-1,-2,-3,-4',  # Use last 4 layers
    subtoken_pooling='first',
    allow_long_sentences=True
)
tagger = SequenceTagger(hidden_size=256,
                         embeddings=embeddings,
                         tag_dictionary=tag_dictionary,
                         tag_type='ner',
                         use_crf=False,
                         use_rnn=False,
                         reproject_embeddings=False,
                         tag_format = 'BIOES')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


2024-11-05 13:48:36,528 SequenceTagger predicts: Dictionary with 45 tags: O, S-SKILL, B-SKILL, E-SKILL, I-SKILL, S-JOB, B-JOB, E-JOB, I-JOB, S-WORK, B-WORK, E-WORK, I-WORK, S-COMPANY, B-COMPANY, E-COMPANY, I-COMPANY, S-LOC, B-LOC, E-LOC, I-LOC, S-UNI, B-UNI, E-UNI, I-UNI, S-NAME, B-NAME, E-NAME, I-NAME, S-PHONE, B-PHONE, E-PHONE, I-PHONE, S-DEG, B-DEG, E-DEG, I-DEG, S-STUDY, B-STUDY, E-STUDY, I-STUDY, S-EMAIL, B-EMAIL, E-EMAIL, I-EMAIL


In [8]:
# train flair ner model
from flair.trainers import ModelTrainer
from flair.training_utils import EvaluationMetric

trainer = ModelTrainer(tagger, corpus)

trainer.train(
    base_path='flair_output/',
    learning_rate=0.01,
    mini_batch_size=8,
    max_epochs=30,
    train_with_dev=False
)
!cp -r ./flair_output /content/drive/MyDrive/FYP/Implementation/

2024-11-05 13:48:40,713 ----------------------------------------------------------------------------------------------------
2024-11-05 13:48:40,716 Model: "SequenceTagger(
  (embeddings): TransformerWordEmbeddings(
    (model): RobertaModel(
      (embeddings): RobertaEmbeddings(
        (word_embeddings): Embedding(50266, 768, padding_idx=1)
        (position_embeddings): Embedding(514, 768, padding_idx=1)
        (token_type_embeddings): Embedding(1, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): RobertaEncoder(
        (layer): ModuleList(
          (0-11): 12 x RobertaLayer(
            (attention): RobertaAttention(
              (self): RobertaSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768

/usr/local/lib/python3.10/dist-packages/flair/trainers/trainer.py:499: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=use_amp and flair.device.type != "cpu")


2024-11-05 13:48:44,919 epoch 1 - iter 3/34 - loss 3.98899651 - time (sec): 4.15 - samples/sec: 1556.43 - lr: 0.010000 - momentum: 0.000000
2024-11-05 13:48:47,896 epoch 1 - iter 6/34 - loss 2.90786798 - time (sec): 7.13 - samples/sec: 1935.27 - lr: 0.010000 - momentum: 0.000000
2024-11-05 13:48:50,572 epoch 1 - iter 9/34 - loss 2.22880260 - time (sec): 9.80 - samples/sec: 2073.66 - lr: 0.010000 - momentum: 0.000000
2024-11-05 13:48:53,517 epoch 1 - iter 12/34 - loss 1.88428828 - time (sec): 12.75 - samples/sec: 2165.05 - lr: 0.010000 - momentum: 0.000000
2024-11-05 13:48:56,211 epoch 1 - iter 15/34 - loss 1.72259978 - time (sec): 15.44 - samples/sec: 2174.03 - lr: 0.010000 - momentum: 0.000000
2024-11-05 13:48:59,039 epoch 1 - iter 18/34 - loss 1.58946207 - time (sec): 18.27 - samples/sec: 2169.91 - lr: 0.010000 - momentum: 0.000000
2024-11-05 13:49:01,602 epoch 1 - iter 21/34 - loss 1.47110825 - time (sec): 20.83 - samples/sec: 2221.07 - lr: 0.010000 - momentum: 0.000000
2024-11-05 1

100%|██████████| 1/1 [00:02<00:00,  2.24s/it]

2024-11-05 13:49:16,449 DEV : loss 0.533987820148468 - f1-score (micro avg)  0.0
2024-11-05 13:49:16,466  - 0 epochs without improvement
2024-11-05 13:49:16,468 ----------------------------------------------------------------------------------------------------


...

100%|██████████| 1/1 [00:01<00:00,  1.80s/it]

2024-11-05 14:06:56,119 DEV : loss 0.46806713938713074 - f1-score (micro avg)  0.0
2024-11-05 14:06:56,137  - 1 epochs without improvement


2024-11-05 14:07:00,015 ----------------------------------------------------------------------------------------------------
2024-11-05 14:07:00,022 Testing using last state of model ...


100%|██████████| 2/2 [00:06<00:00,  3.03s/it]

2024-11-05 14:07:06,143 
Results:
- F-score (micro) 0.4752
- F-score (macro) 0.5364
- Accuracy 0.3191

By class:
              precision    recall  f1-score   support

       SKILL     0.3988    0.2582    0.3135       519
         JOB     0.4972    0.5933    0.5410       150
        WORK     0.6667    0.8819    0.7593       127
     COMPANY     0.4403    0.5842    0.5021       101
         UNI     0.4490    0.4231    0.4356        52
         DEG     0.5000    0.4490    0.4731        49
         LOC     0.7333    0.1467    0.2444        75
        NAME     0.8529    0.8788    0.8657        33
       PHONE     0.8571    0.9375    0.8955        32
       EMAIL     0.8462    0.8148    0.8302        27
       STUDY     0.5000    0.0208    0.0400        48

   micro avg     0.5196    0.4378    0.4752      1213
   macro avg     0.6129    0.5444    0.5364      1213
weighted avg     0.5078    0.4378    0.4425      1213

2024-11-05 14:07:06,145 --------------------------------------------------

In [9]:
# evaluate model
from flair.data import Corpus
from flair.datasets import ColumnCorpus
from flair.models import SequenceTagger
from flair.trainers import ModelTrainer

# Load the trained model
model = SequenceTagger.load('/content/drive/MyDrive/FYP/Implementation/flair_output/final-model.pt')

# Evaluate the model on the test set
result = model.evaluate(corpus.test, gold_label_type='ner', mini_batch_size=32)

# Print the results
# print("Evaluation Loss:", eval_loss)
print(result.detailed_results)  # print the precision, recall, and F1-score per entity type

2024-11-05 14:07:58,501 SequenceTagger predicts: Dictionary with 45 tags: O, S-SKILL, B-SKILL, E-SKILL, I-SKILL, S-JOB, B-JOB, E-JOB, I-JOB, S-WORK, B-WORK, E-WORK, I-WORK, S-COMPANY, B-COMPANY, E-COMPANY, I-COMPANY, S-LOC, B-LOC, E-LOC, I-LOC, S-UNI, B-UNI, E-UNI, I-UNI, S-NAME, B-NAME, E-NAME, I-NAME, S-PHONE, B-PHONE, E-PHONE, I-PHONE, S-DEG, B-DEG, E-DEG, I-DEG, S-STUDY, B-STUDY, E-STUDY, I-STUDY, S-EMAIL, B-EMAIL, E-EMAIL, I-EMAIL


100%|██████████| 3/3 [00:04<00:00,  1.52s/it]


Results:
- F-score (micro) 0.4752
- F-score (macro) 0.5364
- Accuracy 0.3191

By class:
              precision    recall  f1-score   support

       SKILL     0.3988    0.2582    0.3135       519
         JOB     0.4972    0.5933    0.5410       150
        WORK     0.6667    0.8819    0.7593       127
     COMPANY     0.4403    0.5842    0.5021       101
         UNI     0.4490    0.4231    0.4356        52
         DEG     0.5000    0.4490    0.4731        49
         LOC     0.7333    0.1467    0.2444        75
        NAME     0.8529    0.8788    0.8657        33
       PHONE     0.8571    0.9375    0.8955        32
       EMAIL     0.8462    0.8148    0.8302        27
       STUDY     0.5000    0.0208    0.0400        48

   micro avg     0.5196    0.4378    0.4752      1213
   macro avg     0.6129    0.5444    0.5364      1213
weighted avg     0.5078    0.4378    0.4425      1213



In [10]:
# make prediction
import flair
model = SequenceTagger.load('/content/drive/MyDrive/FYP/Implementation/flair_output/best-model.pt')
resume_text = '''
John Doe lives at 1234 Elm Street in Los Angeles, CA 90001. He can be reached at +1 (555) 123-4567 or via email at john.doe@example.com. John is a results-driven software engineer with over 5 years of experience in web development and cloud infrastructure, with strong knowledge of JavaScript, Python, and cloud technologies like AWS and Azure. Currently, he works as a Software Engineer at Google LLC in San Francisco, CA, where he has been employed since August 2019. In this role, he has developed scalable web applications using JavaScript, Node.js, and React, deployed and maintained cloud infrastructure on AWS, reducing downtime by 20%, and led a team of 4 engineers to enhance backend performance by 30%. Previously, he worked as a Junior Developer at Tech Innovators Inc. in Austin, TX, from July 2017 to July 2019, where he created RESTful APIs using Python and Flask, collaborated with front-end developers to build and deploy user-facing applications, and wrote unit and integration tests, improving code coverage by 15%.

John holds a Master of Science in Computer Science from the University of California, Berkeley, with a graduation date of May 2017, and a Bachelor of Science in Information Technology from the University of Texas at Austin, graduated in May 2015. His skillset includes proficiency in programming languages like Python, JavaScript, and Java; frameworks such as React, Flask, and Django; cloud platforms including AWS, Google Cloud, and Azure; as well as other tools like Git, Docker, Kubernetes, and SQL. He is certified as an AWS Certified Solutions Architect – Associate, earned in 2020, and as a Google Professional Cloud Architect, earned in 2021'
'''
sentence = flair.data.Sentence(resume_text.lower())

model.predict(sentence)

print(sentence.to_tagged_string())

2024-11-05 14:08:07,854 SequenceTagger predicts: Dictionary with 47 tags: O, S-SKILL, B-SKILL, E-SKILL, I-SKILL, S-JOB, B-JOB, E-JOB, I-JOB, S-WORK, B-WORK, E-WORK, I-WORK, S-COMPANY, B-COMPANY, E-COMPANY, I-COMPANY, S-LOC, B-LOC, E-LOC, I-LOC, S-UNI, B-UNI, E-UNI, I-UNI, S-DEG, B-DEG, E-DEG, I-DEG, S-NAME, B-NAME, E-NAME, I-NAME, S-PHONE, B-PHONE, E-PHONE, I-PHONE, S-STUDY, B-STUDY, E-STUDY, I-STUDY, S-EMAIL, B-EMAIL, E-EMAIL, I-EMAIL, <START>, <STOP>
Sentence[326]: " john doe lives at 1234 elm street in los angeles, ca 90001. he can be reached at +1 (555) 123-4567 or via email at john.doe@example.com. john is a results-driven software engineer with over 5 years of experience in web development and cloud infrastructure, with strong knowledge of javascript, python, and cloud technologies like aws and azure. currently, he works as a software engineer at google llc in san francisco, ca, where he has been employed since august 2019. in this role, he has developed scalable web applications

In [11]:
from flair.models import SequenceTagger
from flair.data import Sentence

# Load the pretrained NER model
tagger = SequenceTagger.load("/content/drive/MyDrive/FYP/Implementation/flair_output/best-model.pt")
# Example text
text = "Apple is looking at buying U.K. startup for $1 billion."

# Create a Sentence object
sentence = Sentence(resume_text)

# Predict entities
tagger.predict(sentence)

# Print the detected entities
for entity in sentence.get_spans("ner"):
    print(f"Entity: {entity.text}, Type: {entity.get_label('ner').value}, Confidence: {entity.score}")


2024-11-05 14:08:11,394 SequenceTagger predicts: Dictionary with 47 tags: O, S-SKILL, B-SKILL, E-SKILL, I-SKILL, S-JOB, B-JOB, E-JOB, I-JOB, S-WORK, B-WORK, E-WORK, I-WORK, S-COMPANY, B-COMPANY, E-COMPANY, I-COMPANY, S-LOC, B-LOC, E-LOC, I-LOC, S-UNI, B-UNI, E-UNI, I-UNI, S-DEG, B-DEG, E-DEG, I-DEG, S-NAME, B-NAME, E-NAME, I-NAME, S-PHONE, B-PHONE, E-PHONE, I-PHONE, S-STUDY, B-STUDY, E-STUDY, I-STUDY, S-EMAIL, B-EMAIL, E-EMAIL, I-EMAIL, <START>, <STOP>
Entity: Doe lives, Type: NAME, Confidence: 0.16702505946159363
Entity: 1234 Elm, Type: NAME, Confidence: 0.11599351465702057
Entity: Los, Type: DEG, Confidence: 0.18544061481952667
Entity: 90001, Type: DEG, Confidence: 0.14748063683509827
Entity: 555, Type: DEG, Confidence: 0.11017131805419922
Entity: 123-4567, Type: DEG, Confidence: 0.11569119244813919
Entity: via, Type: DEG, Confidence: 0.20629706978797913
Entity: at, Type: EMAIL, Confidence: 0.19365909695625305
Entity: john.doe, Type: JOB, Confidence: 0.10587572306394577
Entity: @, Ty